In [1]:
!pip install transformers sentence-transformers

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\mahno\\Anaconda\\envs\\pytorch_env\\Lib\\site-packages\\~orch\\lib\\asmjit.dll'
Consider using the `--user` option or check the permissions.




  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached torch-1.10.0-cp39-cp39-win_amd64.whl (226.5 MB)
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=ec52a4c5c398556716720ce055bdf95eacfe814ac5af1cf18a1ac71342b7e91a
  Stored in directory: c:\users\mahno\appdata\local\pip\cache\wheels\17\1d\fd\a16123b417c527e0452c3e10ae5139cd2ab2f6fee93e892441
Successfully built sentence-transformers
  Attempting uninstall: torch
    Found existing installation: torch 1.8.0
    Uninstalling torch-1.8.0:
      Successfully uninstalled torch-1.8.0


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from torch import FloatTensor, device
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm


In [3]:
f = open("wordlist-german.txt", "r")
german_words = [word.replace('\n','') for word in f.readlines()]

In [4]:
german_words.sort()

In [18]:


def cos_sim(a: Tensor, b: Tensor):
    """
    borrowed from sentence transformers repo
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('shahrukhx01/paraphrase-mpnet-base-v2-fuzzy-matcher')
model = AutoModel.from_pretrained('shahrukhx01/paraphrase-mpnet-base-v2-fuzzy-matcher')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

##fuzzy_embeddings = model.encode(words)

##print("Fuzzy Match score:")
##print(util.cos_sim(fuzzy_embeddings[0], fuzzy_embeddings[1]))
def get_embedding(word):
  spaced_word = " ".join(list(german_word))
  # Tokenize sentences
  encoded_input = tokenizer([spaced_word], padding=True, truncation=True, return_tensors='pt').to(device)
  # Compute token embeddings
  with torch.no_grad():
      model_output = model(**encoded_input)

  # Perform pooling. In this case, max pooling.
  fuzzy_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])


In [21]:
def vectorize_words(word_list):
  embeddings = []
  for german_word in tqdm(word_list):
    spaced_word = " ".join(list(german_word))
    # Tokenize sentences
    encoded_input = tokenizer([spaced_word], padding=True, truncation=True, return_tensors='pt').to(device)
    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)

    # Perform pooling. In this case, max pooling.
    fuzzy_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

    embeddings.append(fuzzy_embeddings[0])
  return embeddings

In [33]:
german_embeddings = torch.stack(vectorize_words(word_list=german_words[:10000]))

100%|██████████| 10000/10000 [02:23<00:00, 69.51it/s]


In [37]:
german_words[1500]

'Abdeckgitter'

In [41]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from typing import List
import torch
import re

class SentimentModel():
    def __init__(self, model_name: str):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.clean_chars = re.compile(r'[^A-Za-züöäÖÜÄß ]', re.MULTILINE)
        self.clean_http_urls = re.compile(r'https*\\S+', re.MULTILINE)
        self.clean_at_mentions = re.compile(r'@\\S+', re.MULTILINE)

    def predict_sentiment(self, texts: List[str])-> List[str]:
        texts = [self.clean_text(text) for text in texts]
        # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
        input_ids = self.tokenizer(texts, padding=True, truncation=True, add_special_tokens=True)
        input_ids = torch.tensor(input_ids["input_ids"])

        with torch.no_grad():
            logits = self.model(input_ids)    

        label_ids = torch.argmax(logits[0], axis=1)

        labels = [self.model.config.id2label[label_id] for label_id in label_ids.tolist()]
        return labels

    def replace_numbers(self,text: str) -> str:
            return text.replace("0"," null").replace("1"," eins").replace("2"," zwei").replace("3"," drei").replace("4"," vier").replace("5"," fünf").replace("6"," sechs").replace("7"," sieben").replace("8"," acht").replace("9"," neun")         

    def clean_text(self,text: str)-> str:    
            text = text.replace("\n", " ")        
            text = self.clean_http_urls.sub('',text)
            text = self.clean_at_mentions.sub('',text)        
            text = self.replace_numbers(text)                
            text = self.clean_chars.sub('', text) # use only text chars                          
            text = ' '.join(text.split()) # substitute multiple whitespace with single whitespace   
            text = text.strip().lower()
            return text

texts = ["Mit keinem guten Ergebniss","Das war unfair", "Das ist gar nicht mal so gut",
        "Total awesome!","nicht so schlecht wie erwartet", "Das ist gar nicht mal so schlecht",
        "Der Test verlief positiv.","Sie fährt ein grünes Auto.", "Der Fall wurde an die Polzei übergeben."]

model_sentiment = SentimentModel(model_name = "oliverguhr/german-sentiment-bert")

print(model_sentiment.predict_sentiment(texts))

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/161 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/249k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

['negative', 'negative', 'negative', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'neutral']


In [42]:
model_sentiment.predict_sentiment(texts)

MPNetModel(
  (embeddings): MPNetEmbeddings(
    (word_embeddings): Embedding(30527, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): MPNetEncoder(
    (layer): ModuleList(
      (0): MPNetLayer(
        (attention): MPNetAttention(
          (attn): MPNetSelfAttention(
            (q): Linear(in_features=768, out_features=768, bias=True)
            (k): Linear(in_features=768, out_features=768, bias=True)
            (v): Linear(in_features=768, out_features=768, bias=True)
            (o): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (intermediate): MPNetIntermediate(
          (dense): Linear(in_features

In [5]:
len(german_words)

1908815

In [13]:
import torch
from transformers import AutoTokenizer, AutoModel
from torch import Tensor, device

def cos_sim(a: Tensor, b: Tensor):
    """
    borrowed from sentence transformers repo
    Computes the cosine similarity cos_sim(a[i], b[j]) for all i and j.
    :return: Matrix with res[i][j]  = cos_sim(a[i], b[j])
    """
    if not isinstance(a, torch.Tensor):
        a = torch.tensor(a)

    if not isinstance(b, torch.Tensor):
        b = torch.tensor(b)

    if len(a.shape) == 1:
        a = a.unsqueeze(0)

    if len(b.shape) == 1:
        b = b.unsqueeze(0)

    a_norm = torch.nn.functional.normalize(a, p=2, dim=1)
    b_norm = torch.nn.functional.normalize(b, p=2, dim=1)
    return torch.mm(a_norm, b_norm.transpose(0, 1))


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


# Words we want fuzzy embeddings for
word1 = "book"
word1 = " ".join([char for char in word1]) ## divide the word to char level to fuzzy match
word2 = "bo0k"
word2 = " ".join([char for char in word2]) ## divide the word to char level to fuzzy match
words = [word1, word2]
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('shahrukhx01/paraphrase-mpnet-base-v2-fuzzy-matcher')
model = AutoModel.from_pretrained('shahrukhx01/paraphrase-mpnet-base-v2-fuzzy-matcher')

# Tokenize sentences
encoded_input = tokenizer(words, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling. In this case, max pooling.
fuzzy_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

print("Fuzzy Match score:")
print(cos_sim(fuzzy_embeddings[0], fuzzy_embeddings[1]))

Fuzzy Match score:
tensor([[0.7874]])


In [12]:
len(german_words)

1908815

In [14]:
!pip install transformers-interpret

     |████████████████████████████████| 1.4 MB 14.6 MB/s 
  Created wheel for transformers-interpret: filename=transformers_interpret-0.5.2-py3-none-any.whl size=23518 sha256=5e7e9e18ec9cec4f5fe6ce4946f56e2a98a913166a8bb51c2967264c915f4945
  Stored in directory: /root/.cache/pip/wheels/18/8a/5f/db643e1cd44e7bb22c21b52b8fcaebadafaf0b361c33f05869
Successfully built transformers-interpret


In [25]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "oliverguhr/german-sentiment-bert"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# With both the model and tokenizer initialized we are now able to get explanations on an example text.

from transformers_interpret import SequenceClassificationExplainer
cls_explainer = SequenceClassificationExplainer(
    model,
    tokenizer)
word_attributions = cls_explainer("Mullahs sind eine Gefahr für #Europa und für Israel Deutsche Aussenpolitik spielt keine Rolle mehr! HeikoMaas abwählen!")


In [26]:
word_attributions

[('[CLS]', 0.0),
 ('Mul', -0.05583760884139787),
 ('##la', 0.06019875527909861),
 ('##hs', -0.1508126901320417),
 ('sind', -0.16613766576430172),
 ('eine', 0.07261698921953023),
 ('Gefahr', -0.001226666085381778),
 ('für', -0.15111431267775538),
 ('#', -0.03242584554336779),
 ('Europa', -0.011363626393268236),
 ('und', -0.07879812572890692),
 ('für', -0.03801009605975587),
 ('Israel', -0.03199323291376298),
 ('Deutsche', -0.011482409070941522),
 ('Auss', -0.0012122561148097176),
 ('##en', 0.028235223214936964),
 ('##politik', -0.24716960442823183),
 ('spielt', 0.11956056727801602),
 ('keine', 0.29249004190409855),
 ('Rolle', -0.004185719648687352),
 ('mehr', 0.3137312680612055),
 ('!', 0.5950229160712545),
 ('Hei', 0.07600567439136749),
 ('##ko', 0.08617568297775909),
 ('##Ma', -0.0016697434670998824),
 ('##as', -0.3855206775099513),
 ('ab', 0.2695540067378653),
 ('##wä', -0.01402148227621083),
 ('##hlen', 0.02376471294609416),
 ('!', 0.23521338919658957),
 ('[SEP]', 0.0)]

In [31]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from typing import List
import torch
import re

class SentimentModel():
    def __init__(self, model_name: str):
        self.model = AutoModelForSequenceClassification.from_pretrained(model_name)
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)

        self.clean_chars = re.compile(r'[^A-Za-züöäÖÜÄß ]', re.MULTILINE)
        self.clean_http_urls = re.compile(r'https*\\S+', re.MULTILINE)
        self.clean_at_mentions = re.compile(r'@\\S+', re.MULTILINE)

    def predict_sentiment(self, texts: List[str])-> List[str]:
        texts = [self.clean_text(text) for text in texts]
        # Add special tokens takes care of adding [CLS], [SEP], <s>... tokens in the right way for each model.
        input_ids = self.tokenizer(texts, padding=True, truncation=True, add_special_tokens=True)
        input_ids = torch.tensor(input_ids["input_ids"])

        with torch.no_grad():
            logits = self.model(input_ids)    

        label_ids = torch.argmax(logits[0], axis=1)

        labels = [self.model.config.id2label[label_id] for label_id in label_ids.tolist()]
        return labels

    def replace_numbers(self,text: str) -> str:
            return text.replace("0"," null").replace("1"," eins").replace("2"," zwei").replace("3"," drei").replace("4"," vier").replace("5"," fünf").replace("6"," sechs").replace("7"," sieben").replace("8"," acht").replace("9"," neun")         

    def clean_text(self,text: str)-> str:    
            text = text.replace("\n", " ")        
            text = self.clean_http_urls.sub('',text)
            text = self.clean_at_mentions.sub('',text)        
            text = self.replace_numbers(text)                
            text = self.clean_chars.sub('', text) # use only text chars                          
            text = ' '.join(text.split()) # substitute multiple whitespace with single whitespace   
            text = text.strip().lower()
            return text

texts = ["Mullahs sind eine Gefhar für #Europa und for Israel Dustche Aussenpolitik spielt keine Rolle mehr HeikoMaas abwählen!"]

model = SentimentModel(model_name = "oliverguhr/german-sentiment-bert")

print(model.predict_sentiment(texts))

['neutral']
